<a href="https://colab.research.google.com/github/nxxk23/AI-Engineer/blob/main/fraud/data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.0 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
import pandas as pd
bill = pd.read_csv('/content/drive/MyDrive/AIEngineer/fraud/mockup/range_billing_by_icd10.csv')
order = pd.read_csv('/content/drive/MyDrive/AIEngineer/fraud/mockup/range_order_by_icd10.csv')
icd10_df = pd.read_csv('/content/drive/MyDrive/AIEngineer/fraud/icd10 - ICD-10 v.2019.csv')
data = pd.read_csv('/content/drive/MyDrive/AIEngineer/fraud/opd_discharge_August_2024.csv')
data = data[data['order_item']!='[]']
df = data.copy()
data.shape

(16477, 92)

# **extract icd10**

In [11]:
import json

def extract_primary_icd10(diagnosis_str):
    try:
        diagnoses = json.loads(diagnosis_str)
        if isinstance(diagnoses, list):
            for d in diagnoses:
                if isinstance(d, dict) and d.get("dx_type") == "PP":
                    return d.get("icd10")
    except (json.JSONDecodeError, TypeError):
        pass
    return None

data['icd10'] = data['diagnosis'].apply(extract_primary_icd10)
df['icd10'] = df['diagnosis'].apply(extract_primary_icd10)

In [7]:
bill.head(2)

,simb_billing_code,local_billing_name,average_initial_price,billing_count,min_initial_price,max_initial_price,price_list,range,icd10,illness_type
0,1.1.1(3),Outpatient Medication ค่ายาผู้ป่วยนอก,1032.0,17,0.0,3581.0,"[0.0, 68.0, 100.0, 117.0, 170.0, 490.0, 1032.0...",0.0-3581.0,S711,ACC
1,1.1.1(4),ค่าสารอาหารทางเส้นเลือด,116.0,1,116.0,116.0,[116.0],116.0-116.0,S711,ACC


In [8]:
order.head(2)

,simb_billing_code,local_billing_name,item_id,item_name,average_initial_price,order_count,min_initial_price,max_initial_price,price_list,icd10,range
0,1.1.1(3),Outpatient Medication ค่ายาผู้ป่วยนอก,0101H03R0020O01,= ยาเม็ด = Reparil 20 mg #,445.0,1,445.0,445.0,[445.0],S711,445.0-445.0
1,1.1.1(3),Outpatient Medication ค่ายาผู้ป่วยนอก,0104B04T0500O07,Tylenol 650 mg แบบออกฤทธิ์ 8 ชั่วโมง #,281.5,2,188.0,375.0,"[188.0, 375.0]",S711,188.0-375.0


## หาค่ากลางของค่ารักษาตามโรค (ICD-10)
ใช้ค่าเฉลี่ยรวมของค่าใช้จ่ายทั้งหมดจาก `billing_items ` ที่เกี่ยวข้องกับโรคนั้น ๆ ได้ โดยการ**`รวม average_initial_price`** ของ `simb_billing_code` ทั้งหมดใน `billing_summary` ตามแต่ละ ICD-10 รหัส เพื่อให้ได้ราคากลางของแต่ละโรคที่ใกล้เคียงที่สุด

In [14]:
# สร้างตารางสรุปค่าใช้จ่ายกลางของแต่ละโรคตาม ICD-10
icd10_summary = bill.groupby(['icd10', 'illness_type']).agg(
    total_average_price=('average_initial_price', 'sum'),  # รวมค่าเฉลี่ยของค่าบริการทั้งหมดของโรคนั้น
    total_min_price=('min_initial_price', 'sum'),  # รวมค่าต่ำสุดของค่าบริการทั้งหมดของโรคนั้น
    total_max_price=('max_initial_price', 'sum'),  # รวมค่าสูงสุดของค่าบริการทั้งหมดของโรคนั้น
    total_billing_count=('billing_count', 'sum')  # จำนวนรายการรวม
).reset_index()

# คำนวณช่วงของราคากลางสำหรับการอ้างอิง
icd10_summary['price_range'] = icd10_summary['total_min_price'].astype(str) + '-' + icd10_summary['total_max_price'].astype(str)

# แสดงผลลัพธ์ตารางของราคากลางของโรค
display(icd10_summary)

,icd10,illness_type,total_average_price,total_min_price,total_max_price,total_billing_count,price_range
0,A009,ILL,1452.0,1452.0,1452.0,4,1452.0-1452.0
1,A010,ILL,2611.0,2611.0,2611.0,6,2611.0-2611.0
2,A048,ILL,2800.0,2800.0,2800.0,4,2800.0-2800.0
3,A049,ILL,1576.0,1352.0,3349.0,11,1352.0-3349.0
4,A05,ILL,1340.0,1080.0,1600.0,6,1080.0-1600.0
...,...,...,...,...,...,...,...
1878,Z952,ILL,1622.0,1622.0,1622.0,4,1622.0-1622.0
1879,Z955,ILL,1326.0,988.0,2336.0,10,988.0-2336.0
1880,Z961,ACC,1135.5,975.0,1296.0,11,975.0-1296.0
1881,Z99,ILL,700.0,700.0,700.0,2,700.0-700.0


In [46]:
icd10_summary.to_csv('/content/drive/MyDrive/AIEngineer/fraud/mockup/icd10_billprice.csv', index = False)

# **mockup data**

> สุ่มเลือก order_items (item ของค่าการรักษา) โดยให้รวมราคาสุทธิ (net_amount) ของทุก item เข้าด้วยกันแล้วใกล้เคียงกับราคากลาง (total_average_price) ของโรคนั้นที่สุด ซึ่งทำได้โดย:

1. เลือก item อย่างเป็นระบบ: เริ่มจาก**`สุ่ม item แต่ละตัว แล้วค่อยๆ รวมราคาสุทธิจนใกล้เคียงกับราคากลาง`** (ประมาณ total_average_price)
2. คำนวณและปรับราคาโดยให้เป็นไปตามราคากลาง: หากผลรวมราคาสุทธิเกินหรือต่ำกว่าราคากลางไปมาก เราจะ**`ค่อยๆ ปรับ discount เพื่อให้ผลลัพธ์ใกล้เคียงกับราคากลาง`**

In [22]:
import pandas as pd
import random
from faker import Faker
import json

hospital_codes = [11787, 40765, 11583, 11558, 11768, 23781, 41829, 11864, 14683,
                  21673, 11720, 11771, 14591, 11543, 41345, 24258, 11963, 24257,
                  13766, 11645, 14354, 14550, 11766, 23924, 11878, 12026, 41556,
                  12188, 12226, 12159, 13779, 41674, 14641, 14534, 12139, 11765,
                  28875, 42025, 41590, 23875, 11919, 11730, 11537, 10814, 41569,
                  11845, 11708, 11644, 11648, 10686, 44109, 10827, 10734, 10663,
                  10679, 42167, 11874, 42770, 10821, 44353, 42015, 11037, 11656,
                  10695, 12230, 42141, 10849, 11359, 11621, 12212, 13778, 11920,
                  10674, 10723, 10671, 10998, 10822, 10667, 11360, 10832]

In [44]:
# Initial setup
fake = Faker(['en_US', 'th_TH'])
num_rows = 10000

# Generate mock data
data = []
for _ in range(num_rows):
    # Basic patient and doctor info
    hospital_code = random.choice(hospital_codes)
    cid = fake.unique.random_number(digits=13)
    patient = {
        'hn': f"{random.randint(10, 99)}-{random.randint(10000, 99999)}",
        'dob': fake.date_of_birth().isoformat(),
        'gender': random.choice(['M', 'F']),
        'last_name': fake.last_name(),
        'first_name': fake.first_name(),
        'title_name': random.choice(['นาย', 'นาง', 'นางสาว']),
        'patient_registration_id': fake.uuid4()
    }
    vital_sign = [{
        'heart_rate': str(random.randint(60, 100)),
        'pain_score': str(random.randint(0, 10)),
        'systolic_bp': f"{random.uniform(90, 180):.2f}",
        'temperature': f"{random.uniform(36.0, 37.5):.2f}",
        'diastolic_bp': f"{random.uniform(60, 120):.2f}",
        'respiratory_rate': str(random.randint(12, 20)),
        'oxygen_saturation': f"{random.uniform(95, 100):.2f}",
        'vital_sign_entry_datetime': fake.date_time_this_year().strftime("%Y-%m-%d %H:%M")
    }]
    doctor = [{
        'role': 'OWNER',
        'title': 'นพ.',
        'license': str(random.randint(10000, 99999)),
        'lastname': fake.last_name(),
        'firstname': fake.first_name()
    }]

    # Select ICD-10 code and retrieve dx_name and illness details
    diagnosis_row = bill.sample(1).iloc[0]
    icd10_code = diagnosis_row['icd10']
    dx_name_row = icd10_df[icd10_df['ICD-10 Code'] == icd10_code]['Descriptions (EN)']
    dx_name = dx_name_row.values[0] if not dx_name_row.empty else 'Unknown Diagnosis'
    illness_type = diagnosis_row['illness_type']

    # Average price information for the ICD-10 code
    icd10_info = icd10_summary[icd10_summary['icd10'] == icd10_code]
    avg_price_for_icd10 = icd10_info['total_average_price'].values[0]

    # Filter items for the selected ICD-10 code
    matching_order_items = order[order['icd10'] == icd10_code]
    selected_items = []
    total_net_amount = 0  # Total net amount for all items in the bill

    # Randomly select items for the bill, while keeping the total close to avg_price_for_icd10
    while total_net_amount < avg_price_for_icd10 and len(selected_items) < len(matching_order_items):
        item_row = matching_order_items.sample(1).iloc[0]

        # Set price within a realistic range
        min_price = item_row['average_initial_price'] * 0.95
        max_price = item_row['average_initial_price'] * 1.05
        initial_price = random.uniform(min_price, max_price)
        discount = random.uniform(0, initial_price * 0.05)
        net_amount = initial_price - discount

        # Add the item to the selected items list and update the total net amount
        selected_items.append({
            'item_id': item_row['item_id'],
            'item_name': item_row['item_name'],
            'simb_billing_code': item_row['simb_billing_code'],
            'initial': f"{initial_price:.2f}",
            'discount': f"{discount:.2f}",
            'net_amount': f"{net_amount:.2f}",
            'item_amount': '1',
            'local_billing_code': item_row['item_id'],
            'local_billing_name': item_row['local_billing_name']
        })
        total_net_amount += net_amount

        # Stop if the total net amount is close enough to the average price
        if abs(total_net_amount - avg_price_for_icd10) < avg_price_for_icd10 * 0.05:
            break

    # Create the billing list
    billing = [{
        'billing_initial': item['initial'],
        'billing_discount': item['discount'],
        'simb_billing_code': item['simb_billing_code'],
        'billing_net_amount': item['net_amount'],
        'local_billing_code': item['local_billing_code'],
        'local_billing_name': item['local_billing_name']
    } for item in selected_items]

    # Append all data to the data list
    data.append({
        'hospital_code': hospital_code,
        'cid': cid,
        'patient': patient,
        'diagnosis': [{'icd10': icd10_code, 'dx_name': dx_name, 'dx_type': 'PP'}],
        'vital_sign': vital_sign,
        'doctor': doctor,
        'billing': billing,
        'order_item': selected_items,
        'illness_type': illness_type,
        'total_bill_amount': f"{total_net_amount:.2f}"
    })

In [45]:
# Display a sample of the data
mockup_data = pd.DataFrame(data)
display(mockup_data)

,hospital_code,cid,patient,diagnosis,vital_sign,doctor,billing,order_item,illness_type,total_bill_amount
0,21673,2878474290403,"{'hn': '93-98131', 'dob': '1987-01-26', 'gende...","[{'icd10': 'L20', 'dx_name': 'Unknown Diagnosi...","[{'heart_rate': '83', 'pain_score': '2', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '53.39', 'billing_discoun...","[{'item_id': '268', 'item_name': 'Taraxin (10 ...",ILL,2165.51
1,10671,332977283211,"{'hn': '44-51929', 'dob': '1999-07-15', 'gende...","[{'icd10': 'S8351', 'dx_name': 'Avulsion of po...","[{'heart_rate': '73', 'pain_score': '4', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '102.17', 'billing_discou...","[{'item_id': 'DRFEECST', 'item_name': 'ตรวจวิน...",ILL,97.55
2,11621,4564479192186,"{'hn': '43-80965', 'dob': '1912-04-25', 'gende...","[{'icd10': 'M6521', 'dx_name': 'Unknown Diagno...","[{'heart_rate': '94', 'pain_score': '5', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '624.39', 'billing_discou...","[{'item_id': 'DFO', 'item_name': 'ค่าตรวจรักษา...",ILL,1774.48
3,11787,5810650787990,"{'hn': '43-21762', 'dob': '1992-10-19', 'gende...","[{'icd10': 'H66.9', 'dx_name': 'Unknown Diagno...","[{'heart_rate': '73', 'pain_score': '8', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '160.83', 'billing_discou...","[{'item_id': '12226_041123901506012_1010674', ...",ILL,939.40
4,10849,7021525148076,"{'hn': '71-91613', 'dob': '1994-01-08', 'gende...","[{'icd10': 'R05', 'dx_name': 'Cough', 'dx_type...","[{'heart_rate': '60', 'pain_score': '0', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '50.57', 'billing_discoun...","[{'item_id': 'REGFEE2', 'item_name': 'ค่าบริกา...",ILL,3078.90
...,...,...,...,...,...,...,...,...,...,...
9995,10663,7705306383011,"{'hn': '26-33132', 'dob': '1975-05-18', 'gende...","[{'icd10': 'H619', 'dx_name': 'Disorder of ext...","[{'heart_rate': '100', 'pain_score': '10', 'sy...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '129.00', 'billing_discou...","[{'item_id': '140100', 'item_name': 'ค่าบริการ...",ILL,1613.57
9996,42141,4331744977790,"{'hn': '63-11401', 'dob': '1981-10-28', 'gende...","[{'icd10': 'H810', 'dx_name': 'Meniere's disea...","[{'heart_rate': '66', 'pain_score': '3', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '79.33', 'billing_discoun...","[{'item_id': '0104I08D0050L01', 'item_name': '...",ILL,8053.83
9997,13766,5620446849625,"{'hn': '82-66128', 'dob': '2011-05-09', 'gende...","[{'icd10': 'S30', 'dx_name': 'Unknown Diagnosi...","[{'heart_rate': '95', 'pain_score': '5', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '84.58', 'billing_discoun...","[{'item_id': '139', 'item_name': '10% Betadine...",ACC,3784.93
9998,23875,4484925120900,"{'hn': '99-74385', 'dob': '1974-11-26', 'gende...","[{'icd10': 'M6268', 'dx_name': 'Muscle strain:...","[{'heart_rate': '72', 'pain_score': '9', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '40.70', 'billing_discoun...","[{'item_id': '0104C14I04', 'item_name': 'IBUPR...",ILL,5264.29


In [47]:
import json

# ฟังก์ชันที่ใช้ในการดึงรหัส ICD-10 จาก `diagnosis` ซึ่งเป็นลิสต์
def extract_primary_icd10(diagnosis_list):
    try:
        # หาก diagnosis_list เป็นลิสต์
        if isinstance(diagnosis_list, list):
            for d in diagnosis_list:
                # ตรวจสอบว่าเป็น dictionary และหาค่าของ icd10
                if isinstance(d, dict) and 'icd10' in d:
                    return d.get("icd10")
    except (json.JSONDecodeError, TypeError):
        pass
    return None

# สมมติว่า mockup_data คือ DataFrame ที่มีคอลัมน์ 'diagnosis' ซึ่งเป็นลิสต์
mockup_data['icd10'] = mockup_data['diagnosis'].apply(extract_primary_icd10)

## **check**

In [61]:
icd10_summary[icd10_summary['icd10']=='W260']

,icd10,illness_type,total_average_price,total_min_price,total_max_price,total_billing_count,price_range
1799,W260,ILL,370.0,370.0,370.0,2,370.0-370.0


In [66]:
order[order['icd10'] == 'W260']

,simb_billing_code,local_billing_name,item_id,item_name,average_initial_price,order_count,min_initial_price,max_initial_price,price_list,icd10,range
57096,1.1.14(1),Packaged Medical Charges ค่าบริการชุดเหมาจ่าย,DRES,DRESSING กลาง[OPD][,300.0,1,300.0,300.0,[300.0],W260,300.0-300.0
57097,1.1.2(1),Medical Supplies 1 เวชภัณฑ์1,SUP1,EASI - FIX COHESIVE,70.0,1,70.0,70.0,[70.0],W260,70.0-70.0


In [68]:
mockup_data[mockup_data['icd10']=='W260']

,hospital_code,cid,patient,diagnosis,vital_sign,doctor,billing,order_item,illness_type,total_bill_amount,icd10
1580,11359,1950863445208,"{'hn': '93-62322', 'dob': '2012-01-03', 'gende...","[{'icd10': 'W260', 'dx_name': 'Contact with ot...","[{'heart_rate': '86', 'pain_score': '3', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '68.20', 'billing_discoun...","[{'item_id': 'SUP1', 'item_name': 'EASI - FIX ...",ILL,138.82,W260
4470,14354,7274420394735,"{'hn': '60-75164', 'dob': '1936-08-09', 'gende...","[{'icd10': 'W260', 'dx_name': 'Contact with ot...","[{'heart_rate': '99', 'pain_score': '4', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '303.83', 'billing_discou...","[{'item_id': 'DRES', 'item_name': 'DRESSING ก...",ILL,585.77,W260
6616,11583,1023614386745,"{'hn': '65-36886', 'dob': '1978-05-22', 'gende...","[{'icd10': 'W260', 'dx_name': 'Contact with ot...","[{'heart_rate': '80', 'pain_score': '9', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '72.76', 'billing_discoun...","[{'item_id': 'SUP1', 'item_name': 'EASI - FIX ...",ILL,373.63,W260
7309,12139,1530265807168,"{'hn': '68-23047', 'dob': '1940-01-23', 'gende...","[{'icd10': 'W260', 'dx_name': 'Contact with ot...","[{'heart_rate': '96', 'pain_score': '5', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '67.36', 'billing_discoun...","[{'item_id': 'SUP1', 'item_name': 'EASI - FIX ...",ILL,361.27,W260
8891,11920,4070819953210,"{'hn': '54-16708', 'dob': '1909-07-22', 'gende...","[{'icd10': 'W260', 'dx_name': 'Contact with ot...","[{'heart_rate': '72', 'pain_score': '0', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '71.22', 'billing_discoun...","[{'item_id': 'SUP1', 'item_name': 'EASI - FIX ...",ILL,371.14,W260


In [65]:
mockup_data[mockup_data['icd10']=='W260']['order_item'].iloc[1]

[{'item_id': 'DRES',
  'item_name': 'DRESSING  กลาง[OPD][',
  'simb_billing_code': '1.1.14(1)',
  'initial': '303.83',
  'discount': '12.04',
  'net_amount': '291.80',
  'item_amount': '1',
  'local_billing_code': 'DRES',
  'local_billing_name': 'Packaged Medical Charges  ค่าบริการชุดเหมาจ่าย '},
 {'item_id': 'DRES',
  'item_name': 'DRESSING  กลาง[OPD][',
  'simb_billing_code': '1.1.14(1)',
  'initial': '296.64',
  'discount': '2.67',
  'net_amount': '293.98',
  'item_amount': '1',
  'local_billing_code': 'DRES',
  'local_billing_name': 'Packaged Medical Charges  ค่าบริการชุดเหมาจ่าย '}]

In [69]:
mockup_data.to_csv('/content/drive/MyDrive/AIEngineer/fraud/mockup/mockup.csv', index=False)

In [70]:
mockup_data

,hospital_code,cid,patient,diagnosis,vital_sign,doctor,billing,order_item,illness_type,total_bill_amount,icd10
0,21673,2878474290403,"{'hn': '93-98131', 'dob': '1987-01-26', 'gende...","[{'icd10': 'L20', 'dx_name': 'Unknown Diagnosi...","[{'heart_rate': '83', 'pain_score': '2', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '53.39', 'billing_discoun...","[{'item_id': '268', 'item_name': 'Taraxin (10 ...",ILL,2165.51,L20
1,10671,332977283211,"{'hn': '44-51929', 'dob': '1999-07-15', 'gende...","[{'icd10': 'S8351', 'dx_name': 'Avulsion of po...","[{'heart_rate': '73', 'pain_score': '4', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '102.17', 'billing_discou...","[{'item_id': 'DRFEECST', 'item_name': 'ตรวจวิน...",ILL,97.55,S8351
2,11621,4564479192186,"{'hn': '43-80965', 'dob': '1912-04-25', 'gende...","[{'icd10': 'M6521', 'dx_name': 'Unknown Diagno...","[{'heart_rate': '94', 'pain_score': '5', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '624.39', 'billing_discou...","[{'item_id': 'DFO', 'item_name': 'ค่าตรวจรักษา...",ILL,1774.48,M6521
3,11787,5810650787990,"{'hn': '43-21762', 'dob': '1992-10-19', 'gende...","[{'icd10': 'H66.9', 'dx_name': 'Unknown Diagno...","[{'heart_rate': '73', 'pain_score': '8', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '160.83', 'billing_discou...","[{'item_id': '12226_041123901506012_1010674', ...",ILL,939.40,H66.9
4,10849,7021525148076,"{'hn': '71-91613', 'dob': '1994-01-08', 'gende...","[{'icd10': 'R05', 'dx_name': 'Cough', 'dx_type...","[{'heart_rate': '60', 'pain_score': '0', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '50.57', 'billing_discoun...","[{'item_id': 'REGFEE2', 'item_name': 'ค่าบริกา...",ILL,3078.90,R05
...,...,...,...,...,...,...,...,...,...,...,...
9995,10663,7705306383011,"{'hn': '26-33132', 'dob': '1975-05-18', 'gende...","[{'icd10': 'H619', 'dx_name': 'Disorder of ext...","[{'heart_rate': '100', 'pain_score': '10', 'sy...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '129.00', 'billing_discou...","[{'item_id': '140100', 'item_name': 'ค่าบริการ...",ILL,1613.57,H619
9996,42141,4331744977790,"{'hn': '63-11401', 'dob': '1981-10-28', 'gende...","[{'icd10': 'H810', 'dx_name': 'Meniere's disea...","[{'heart_rate': '66', 'pain_score': '3', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '79.33', 'billing_discoun...","[{'item_id': '0104I08D0050L01', 'item_name': '...",ILL,8053.83,H810
9997,13766,5620446849625,"{'hn': '82-66128', 'dob': '2011-05-09', 'gende...","[{'icd10': 'S30', 'dx_name': 'Unknown Diagnosi...","[{'heart_rate': '95', 'pain_score': '5', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '84.58', 'billing_discoun...","[{'item_id': '139', 'item_name': '10% Betadine...",ACC,3784.93,S30
9998,23875,4484925120900,"{'hn': '99-74385', 'dob': '1974-11-26', 'gende...","[{'icd10': 'M6268', 'dx_name': 'Muscle strain:...","[{'heart_rate': '72', 'pain_score': '9', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '40.70', 'billing_discoun...","[{'item_id': '0104C14I04', 'item_name': 'IBUPR...",ILL,5264.29,M6268
